In [ ]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer

/home/cosmoquester/aibookathon/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [ ]:
# 학습에 사용했던 요약문을 생성할 때와 똑같은 요약 모델과 생성 옵션을 사용하는 것이 좋습니다.
summarize_model_name: str = "digit82/kobart-summarization"
summarize_num_beams: int = 4
sum_input_max_length: int = 512
sum_max_token_length: int = 128

model_name: str = "khu-bot/polyglot-essayist-with-sum"
tokenizer_name: str = model_name
# Private으로 huggingface model hub에 올라가 있는 경우에 필요합니다.
auth_token: str = None

prompt_max_length = 128
max_length: int = 512
device: str = "cuda"

# 이 개수만큼의 토큰을 생성하면 종료됩니다.
num_generate_tokens: int = 5000

# Load Models

In [ ]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_auth_token=auth_token, **tokenizer_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=auth_token).to(device)

In [ ]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
summarize_tokenizer = AutoTokenizer.from_pretrained(summarize_model_name, use_auth_token=auth_token, **tokenizer_kwargs)
summarize_model = AutoModelForSeq2SeqLM.from_pretrained(summarize_model_name).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# Start Prompt

In [ ]:
# 생성하고 싶은 제목들을 입력해줍니다. 같은 제목을 여러 개 넣어도 다른 두 글이 생성됩니다.
titles = [
    "나의 빛과 어둠",
    "어느 날의 일기",
]

# Write

모델이 출력하는 문장에는 이상한 말들도 섞여있지만 그런 말이 나왔어도 또 그 뒤에 다시 괜찮은 말이 나오기 때문에 생성이 끝난 후 사람이 이상한 텍스트를 없애줄 필요가 있습니다.

In [ ]:
def datum_to_string(datum, use_n_summary: int = 10):
    text = f"제목: {datum['title']}\n"

    summarizations = datum.get("summarizations")
    if summarizations:
        summarizations = summarizations[-use_n_summary:]
        summarization = " ".join(summarizations).replace("\n", " ")
        text = f"요약: {summarization}\n" + text
    return text


In [ ]:
input_examples = [
    {"title": title, "summarizations": [], "content": ""} for title in titles
]

total_tokens = 0

while total_tokens < num_generate_tokens:
  prompts = [datum_to_string(example) for example in input_examples]
  input_ids = tokenizer(
          prompts,
          add_special_tokens=False,
          max_length=prompt_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  outputs = model.generate(input_ids, 
      max_length, 
      do_sample=True,
      num_return_sequences=1,
      pad_token_id=tokenizer.pad_token_id,
      use_cache=True,
  )
  outputs = outputs[:, input_ids.size(1):]
  total_tokens += outputs.size(1)

  texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  # Remove last incomplete sentence
  texts = [text[:max(text.rfind(last_char) for last_char in ".?!\n")] for text in texts]
  for example, text in zip(input_examples, texts):
    example["content"] += text

  # 이 부분은 다른 요약모델을 사용하면 그에 따라 바뀔 필요가 있습니다!
  sum_input_texts = [summarize_tokenizer.bos_token + text + summarize_tokenizer.eos_token for text in texts]
  sum_input_ids = summarize_tokenizer(
          texts,
          add_special_tokens=False,
          max_length=sum_input_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  sum_outputs = summarize_model.generate(sum_input_ids, 
      sum_max_token_length, 
      num_beams=summarize_num_beams,
      num_return_sequences=1,
      eos_token_id=summarize_tokenizer.eos_token_id,
      pad_token_id=summarize_tokenizer.pad_token_id,
      use_cache=True,
  )

  summarizations = summarize_tokenizer.batch_decode(sum_outputs, skip_special_tokens=True)

  for example, summarization in zip(input_examples, summarizations):
    example["summarizations"].append(summarization)

# Print Contents

In [ ]:
for example in input_examples:
    print(f'제목: {example["title"]}\n\n')
    print(example["content"])
    print("\n------------------ END ------------------\n\n")

제목: 나의 빛과 어둠


1
"누나는 무슨 색인데 그 색이야?"
"음.. 나는 짙은 파란색인데!"
"네가 말한 파란색은 어디에서, 어떻게 생겨났어?"
처음 나를 지켜본 친구와 동생이 물어본다. 나는 늘 짙은 파란색이다. 그런데 왜 그런 색이 됐냐고? 생각해보면 그렇다. 나는 왜 파란색일까?
태어난 계절이 겨울과 여름의 경계선인 11월이다. 그러니까 '나'라는 사람은 겨울에서 태어난 것이다. 사람들마다 태어나는 계절과 이유가 다르겠지만, 나에겐 정말 그랬다.
태어난 시기를 생각하면 나의 어린 시절이 떠올라 조금 씁쓸한 마음이 든다. 나의 부모님은 두 분 다 교사였지만, 교대에 다니고 있던 엄마는 학교로 발령받기를 희망했다. 아빠에게는 고민이 많았다. 당장 교대에 붙더라도 남은 임용고시 시험에 낙방을 거듭했다. 아빠도 교사를 꿈꿨다. 하지만 아빠의 꿈을 이뤄줄 수 없을 만큼 그 시절의 교대생은 경쟁률이 치열했다. 엄마는 그런 상황에서도 내가 이 나라에서 교사가 되길 원했다.
그런데 참 아이러니하게도 아빠는 이 모든 상황이 불만이었다. 교대생들은 교사를 되고 싶지만 임용고시에 떨어지는 경우가 많아서 고민의 고민을 거듭했다. 자신의 이상을 펼칠 수 없는 가족과 그 가족에게 희생하는 아내의 삶이 아빠의 인생이었다. 그로 인해 아빠는 누구보다 열심히 일했고, 그 결과로 아빠와 동생을 교사로 만들었다.[부산 여행기록] 해운대에서 만나는 부산여행 코스1(2019.05)_해운대 아쿠아리움::혼자의 힐링 여행_해운대,광안리의 시간#05. 부산을 여행하는 기분을 그대로 느껴요.﻿오랜만에 고등학교 친구들 사이에서의 불금, 해운대에서의 약속을 가는 지하철 안에서 갑작스럽게 정했다.
﻿이 날은, 비가 온 후 다시 폭염주의보가 내려졌다.
그래서, 나는 비가 오는 광안리를 걷기로 했다2년 정도 지나 나는 대학을 졸업했고 첫 직장에 취업했다. 나에게는 정말이지 정말이지 꿈같은 일이 일어났다. 누군가 나의 이름 앞에 이런 직업이 있다고, 이렇게 능력이 출중한 사람들이 있다고 적혀있다면 나는

# Print Summaries

In [ ]:
for example in input_examples:
    print(f'제목: {example["title"]}')
    for i, summary in enumerate(example["summarizations"], start=1):
        print(f"[{i}] {summary}")
    print()

제목: 나의 빛과 어둠
[1] 교대에 다니고 있던 엄마는 학교로 발령받기를 희망했지만 교대에 다니고 있던 엄마는 학교로 발령받기를 희망했으나 교대생들은 교사가 되고 싶지만 임용고시에 떨어지는 경우가 많아서 고민의 고민을 거듭했다.
[2] 대학교를 다닌 당시 내가 가장 후회하는 것 중 하나가 그 꿈 때문에 나의 첫 진로를 내가 선택하지 않았고, 선택할 수가 없었다는 것을 뒤늦게 깨달아 대학을 다니면서 내가 가장 후회했던 점을 다시 선택할 수 없었던 이유를 곰곰이 생각하고 이유를 찾기위해 이유를 찾기위해 노력했다.
[3] 임용고시 공부를 했었던 선배와 수업을 듣기 시작한지 몇 주 후 나는 다시 임용고시학원으로 돌아갔다.
[4] 나는 나의 나를 존중하지 않았고, 나의 주장을 하기에 항상 망설여졌으며, 내가 아닌 다른 생각으로 나를 몰아붙이기 바빴다.
[5] 나 자신을 나 자신으로, 나라는 사람으로 살아내기도 힘든 나인데, 그런 나를 자꾸만 밀어내는 사람을 만나면서 나의 빛이 다시 흐려졌다.
[6] 나의 빛과 어둠은 다른 것이 아닌, 나에게서 비롯된 것이며 빛과 어둠은 다른 것이 아닌, 나에게서 비롯된 것이며 빛과 어둠은 다른 것이 아닌, 나에게서 비롯된 것이며 빛과 어둠은 다른 것이 아닌, 나에게서 비롯된 것이며 빛과 어둠은 다른 것이 아닌, 나에게서 비롯된 것이다.
[7] 나의 빛과 어둠을 응시하는 일, 내가 내 자신을 바라보는 일, 이것이야말로 나를 나로 바라보는 일일 테고, 나의 빛과 어둠을 응시하는 일, 이것이야말로 나를 나로 바라보는 일일 테고, 나의 빛과 어둠을 응시하는 일, 이것이야말로 나를 나로 바라보는 일일 테고, 나의 빛과 어둠을 응시하는 일, 이것이야말로 나를 나로 바라보는 일일 테고, 나의 빛과 어둠을 응시하는 일, 그것이야말로 나를 나로 바라보는 일일 테고, 나의 빛과 어둠을 응시하는 일, 이것이야말로 나를 나로 바라보는 일일 테고, 나를 나로 바라보는 일, 이제 조금은 알겠다.
[8] 나는 내가 쓴 글을 읽자마자 그에게 보내고 싶다.
[9]